In [9]:
# ! pip install scikit-learn

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 45.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.7/37.7 MB 32.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 28.5 MB/s eta 0:00:00
DEPRECATION: lmdeploy 0.1.0-git782048c.abi0.dtk2404.torch2.1. has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of lmdeploy or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: mmcv 2.0.1-gitc0ccf15.abi0.dtk2404.torch2.1. has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of mmcv or contact the author to suggest that they release a version with a conforming version number. Discus

In [2]:
import pandas as pd
lab_pivot = pd.read_csv("lab_pivot.csv")
vital_pivot = pd.read_csv("vital_pivot.csv")
cohort_df = pd.read_csv("cohort_df.csv")


In [3]:
features_df = (
    cohort_df
    .merge(lab_pivot, on='stay_id', how='left')
    .merge(vital_pivot, on='stay_id', how='left')
)


In [4]:
missing_pct = features_df.isnull().mean().sort_values(ascending=False)
print(missing_pct)


Weight        0.987143
BG            0.665714
Hb            0.058571
WBC           0.057143
PLT           0.053571
Ca            0.022143
Phos          0.020714
Glucose       0.014286
AG            0.014286
BUN           0.014286
Scr           0.013571
Cl            0.013571
K             0.012857
Na            0.012857
aki_label     0.009286
subject_id    0.000000
gender        0.000000
age           0.000000
hadm_id       0.000000
stay_id       0.000000
dtype: float64


In [5]:
high_missing_cols = missing_pct[missing_pct > 0.2].index.tolist()
features_df = features_df.drop(columns=high_missing_cols)
print(f"删除变量：{high_missing_cols}")


删除变量：['Weight', 'BG']


In [6]:
features_df['gender'] = features_df['gender'].map({'M': 1, 'F': 0})


In [ ]:
# from sklearn.impute import SimpleImputer

# imputer = SimpleImputer(strategy='median')
X = features_df.drop(columns=['subject_id', 'hadm_id', 'stay_id', 'aki_label'])  # 特征列
# X_imputed = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)


In [11]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=5)
X = features_df.drop(columns=['subject_id', 'hadm_id', 'stay_id', 'aki_label'])  # 特征列
X_imputed = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)


In [12]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X_imputed), columns=X_imputed.columns)


In [14]:
# y = features_df['aki_label']

# from sklearn.model_selection import train_test_split

# features_df = features_df.dropna(subset=['aki_label'])  # 删除 aki_label 缺失的患者


# X_train, X_val, y_train, y_val = train_test_split(
#     X_scaled, y, test_size=0.15, random_state=42, stratify=y
# )


In [15]:
# 删除 aki_label 缺失的行
features_df = features_df.dropna(subset=['aki_label'])

# 提取标签
y = features_df['aki_label'].astype(int)  # 保证为整数（0/1）

# 拆分训练集和验证集
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    X_scaled.loc[features_df.index],  # 保持索引一致
    y,
    test_size=0.15,
    random_state=42,
    stratify=y
)


In [16]:
print(f"训练集样本数: {len(X_train)}")
print(f"验证集样本数: {len(X_val)}")
print("标签分布（训练）:\n", y_train.value_counts(normalize=True))
print("标签分布（验证）:\n", y_val.value_counts(normalize=True))


训练集样本数: 1178
验证集样本数: 209
标签分布（训练）:
 aki_label
0    0.668081
1    0.331919
Name: proportion, dtype: float64
标签分布（验证）:
 aki_label
0    0.669856
1    0.330144
Name: proportion, dtype: float64


In [17]:
X_train.to_csv("X_train.csv", index=False)
X_val.to_csv("X_val.csv", index=False)
y_train.to_csv("y_train.csv", index=False)
y_val.to_csv("y_val.csv", index=False)
